# DuckDuckGoSearch

This notebook provides a quick overview for getting started with [DuckDuckGoSearch](/docs/integrations/tools/). For detailed documentation of all DuckDuckGoSearch features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_community_tools_duckduckgo_search.DuckDuckGoSearch.html).

DuckDuckGoSearch offers a privacy-focused search API designed for LLM Agents. It provides seamless integration with a wide range of data sources, prioritizing user privacy and relevant search results.

## Overview

### Integration details

| Class | Package | Serializable | [PY support](https://python.langchain.com/docs/integrations/tools/ddg/) |  Package latest |
| :--- | :--- | :---: | :---: | :---: |
| [DuckDuckGoSearch](https://api.js.langchain.com/classes/langchain_community_tools_duckduckgo_search.DuckDuckGoSearch.html) | [@langchain/community](https://api.js.langchain.com/modules/langchain_community_tools_duckduckgo_search.html) | beta | ✅ |  ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

### Tool features

- TODO: Add feature table if it makes sense


## Setup

The integration lives in the `@langchain/community` package, along with the `duck-duck-scrape` dependency:

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community duck-duck-scrape
</Npm2Yarn>
```

### Credentials

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

```typescript
process.env.LANGCHAIN_TRACING_V2="true"
process.env.LANGCHAIN_API_KEY="your-api-key"
```

## Instantiation

Here we show how to insatiate an instance of the `DuckDuckGoSearch` tool, with 

In [1]:
import { DuckDuckGoSearch } from "@langchain/community/tools/duckduckgo_search"

const tool = new DuckDuckGoSearch({ maxResults: 1 })

## Invocation

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)

In [2]:
await tool.invoke("What is Anthropic's estimated revenue for 2024?")

[{"title":"Anthropic forecasts more than $850 mln in annualized revenue rate by ...","link":"https://www.reuters.com/technology/anthropic-forecasts-more-than-850-mln-annualized-revenue-rate-by-2024-end-report-2023-12-26/","snippet":"Dec 26 (Reuters) - Artificial intelligence startup <b>Anthropic</b> has projected it will generate more than $850 million in annualized <b>revenue</b> by the end of <b>2024</b>, the Information reported on Tuesday ..."}]


### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

We can also invoke the tool with a model-generated `ToolCall`, in which case a `ToolMessage` will be returned:

In [3]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    input: "What is Anthropic's estimated revenue for 2024?"
  },
  id: "tool_call_id",
  name: tool.name,
  type: "tool_call",
}
await tool.invoke(modelGeneratedToolCall)

ToolMessage {
  "content": "[{\"title\":\"Anthropic forecasts more than $850 mln in annualized revenue rate by ...\",\"link\":\"https://www.reuters.com/technology/anthropic-forecasts-more-than-850-mln-annualized-revenue-rate-by-2024-end-report-2023-12-26/\",\"snippet\":\"Dec 26 (Reuters) - Artificial intelligence startup <b>Anthropic</b> has projected it will generate more than $850 million in annualized <b>revenue</b> by the end of <b>2024</b>, the Information reported on Tuesday ...\"}]",
  "name": "duckduckgo-search",
  "additional_kwargs": {},
  "response_metadata": {},
  "tool_call_id": "tool_call_id"
}


## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [4]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
})

In [12]:
import { ChatPromptTemplate } from "@langchain/core/prompts"
import { RunnableConfig } from "@langchain/core/runnables"
import { AIMessage } from "@langchain/core/messages"

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["human", "{user_input}"],
    ["placeholder", "{messages}"],
  ]
)

// specifying tool_choice will force the model to call this tool.
const llmWithTools = llm.bindTools([tool], {
  tool_choice: tool.name
})

const llmChain = prompt.pipe(llmWithTools);

const toolChain = async (userInput: string, config?: RunnableConfig): Promise<AIMessage> => {
  const input_ = { user_input: userInput };
  const aiMsg = await llmChain.invoke(input_, config);
  const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
  return llmChain.invoke({ ...input_, messages: [aiMsg, ...toolMsgs] }, config);
};

const toolChainResult = await toolChain("What is Anthropic's estimated revenue for 2024?");

In [13]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content
}, null, 2))

AIMessage {
  "tool_calls": [
    {
      "name": "duckduckgo-search",
      "args": {
        "input": "Anthropic revenue 2024 forecast"
      },
      "type": "tool_call",
      "id": "call_E22L1T1bI6xPrMtL8wrKW5C5"
    }
  ],
  "content": ""
}


## With an agent

We can also pass the `DuckDuckGoSearch` tool to an agent:

In [14]:
import { DuckDuckGoSearch } from "@langchain/community/tools/duckduckgo_search";
import { ChatOpenAI } from "@langchain/openai";
import type { ChatPromptTemplate } from "@langchain/core/prompts";

import { pull } from "langchain/hub";
import { AgentExecutor, createOpenAIFunctionsAgent } from "langchain/agents";

// Define the tools the agent will have access to.
const toolsForAgent = [new DuckDuckGoSearch({ maxResults: 1 })];

// Get the prompt to use - you can modify this!
// If you want to see the prompt in full, you can at:
// https://smith.langchain.com/hub/hwchase17/openai-functions-agent
const promptForAgent = await pull<ChatPromptTemplate>(
  "hwchase17/openai-functions-agent"
);
const llmForAgent = new ChatOpenAI({
  model: "gpt-4-turbo-preview",
  temperature: 0,
});
const agent = await createOpenAIFunctionsAgent({
  llm: llmForAgent,
  tools: toolsForAgent,
  prompt: promptForAgent,
});
const agentExecutor = new AgentExecutor({
  agent,
  tools: toolsForAgent,
});
const agentResult = await agentExecutor.invoke({
  input: "What is Anthropic's estimated revenue for 2024?",
});

console.log(agentResult);

{
  input: "What is Anthropic's estimated revenue for 2024?",
  output: 'Anthropic has projected that it will generate more than $850 million in annualized revenue by the end of 2024.'
}


## API reference

For detailed documentation of all DuckDuckGoSearch features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_community_tools_duckduckgo_search.DuckDuckGoSearch.html)